# Quantum ESPRESSO Workshop - Part 5: Ground State Structure Optimization

## Learning Objectives
1. Understand why structure optimization is essential in DFT calculations
2. Master ionic relaxation (`relax`) and variable-cell relaxation (`vc-relax`)
3. Implement the Equation of State (EOS) method for robust volume optimization
4. Fit the Birch-Murnaghan equation and extract bulk modulus
5. Establish a complete workflow for ground state structure determination

---

## 1. Why Structure Optimization is Required

### The Fundamental Problem

When you obtain a crystal structure from a database (Materials Project, ICSD, COD), that structure is typically:
- An **experimental** structure determined by X-ray or neutron diffraction
- A structure optimized with a **different** DFT functional
- A structure from a **different** computational setup (pseudopotentials, cutoffs)

### Why This Matters

**Different exchange-correlation functionals give different equilibrium structures:**

| Functional | Typical Lattice Error | Typical Bulk Modulus Error |
|------------|----------------------|---------------------------|
| LDA        | -1 to -2% (underestimates) | +5 to +15% (overestimates) |
| PBE (GGA)  | +1 to +2% (overestimates) | -5 to -10% (underestimates) |
| PBEsol     | ~0.5% | ~5% |
| SCAN       | <0.5% | <5% |

**Using a non-equilibrium structure introduces systematic errors:**
- Internal stress affects all calculated properties
- Band gaps, phonon frequencies, elastic constants will be wrong
- Results are not self-consistent with your chosen methodology

### The Golden Rule

> **Always optimize the structure with YOUR chosen functional and pseudopotentials before calculating any properties.**

---

## 2. Setup and Configuration

In [ ]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.optimize import curve_fit
import re
import time
import json

# Configure matplotlib for publication-quality figures
plt.rcParams.update({
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 16,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 11,
    'figure.figsize': (10, 6),
    'figure.dpi': 100,
    'savefig.dpi': 150,
    'savefig.bbox': 'tight'
})

# Paths - Adjust these for your system
WORKSHOP_ROOT = Path('/home/niel/git/DFT_Tutorial/qe_workshop_complete')
PSEUDO_DIR = WORKSHOP_ROOT / 'pseudopotentials'
OUTPUT_DIR = WORKSHOP_ROOT / 'outputs'

# Working directory for this notebook
WORK_DIR = OUTPUT_DIR / '05_structure_optimization'
WORK_DIR.mkdir(parents=True, exist_ok=True)
(WORK_DIR / 'tmp').mkdir(exist_ok=True)

print(f"Workshop root: {WORKSHOP_ROOT}")
print(f"Working directory: {WORK_DIR}")
print(f"Pseudopotential directory: {PSEUDO_DIR}")

In [ ]:
# Load converged parameters from previous notebooks (or use defaults)
params_file = WORKSHOP_ROOT / 'converged_parameters.json'

if params_file.exists():
    with open(params_file, 'r') as f:
        params = json.load(f)
    ecutwfc = params.get('ecutwfc_recommended', 40.0)
    ecutrho_factor = params.get('ecutrho_factor', 8)
    kgrid = params.get('kpoints_recommended', 8)
    print("Loaded converged parameters from previous notebooks:")
else:
    ecutwfc = 40.0
    ecutrho_factor = 8
    kgrid = 8
    print("Using default parameters (run convergence tests for production!):")

ecutrho = ecutwfc * ecutrho_factor

print(f"  ecutwfc = {ecutwfc} Ry")
print(f"  ecutrho = {ecutrho} Ry ({ecutrho_factor}x ecutwfc)")
print(f"  k-points = {kgrid}x{kgrid}x{kgrid}")

---

## 3. Types of Optimization in Quantum ESPRESSO

QE provides two main optimization modes:

### 3.1 Ionic Relaxation: `calculation = 'relax'`

**What it does:** Optimizes atomic positions while keeping the unit cell fixed.

**When to use:**
- Optimizing internal coordinates at a known lattice parameter
- Surface or defect calculations with fixed bulk lattice
- After determining optimal cell volume via EOS

**Key parameters:**
- `etot_conv_thr`: Total energy convergence threshold (default: 1e-4 Ry)
- `forc_conv_thr`: Force convergence threshold (default: 1e-3 Ry/Bohr)

### 3.2 Variable-Cell Relaxation: `calculation = 'vc-relax'`

**What it does:** Optimizes both atomic positions AND cell parameters (shape and volume).

**When to use:**
- Finding equilibrium structure from scratch
- When you don't know the lattice parameters
- Phase transition studies

**Key parameters:**
- All `relax` parameters, plus:
- `cell_dofree`: Which cell degrees of freedom to optimize
- `press`: Target pressure (kbar)
- `press_conv_thr`: Pressure convergence threshold (kbar)

### 3.3 `cell_dofree` Options

| Value | Description |
|-------|-------------|
| `'all'` | All cell parameters free (default) |
| `'ibrav'` | Cell shape determined by ibrav, only celldm values change |
| `'x'` | Only a1 vector free |
| `'y'` | Only a2 vector free |
| `'z'` | Only a3 vector free |
| `'xy'` | a1 and a2 free |
| `'xz'` | a1 and a3 free |
| `'yz'` | a2 and a3 free |
| `'xyz'` | All vectors free (equivalent to 'all' with ibrav=0) |
| `'shape'` | Cell shape free, volume fixed |
| `'volume'` | Volume free, shape fixed |
| `'2Dxy'` | For 2D systems in xy plane |

---

## 4. Core Functions for Structure Optimization

In [ ]:
def generate_relax_input(prefix, ecutwfc, ecutrho, kpoints, pseudo_dir,
                         celldm1, calculation='relax', 
                         etot_conv_thr=1.0e-5, forc_conv_thr=1.0e-4,
                         cell_dofree='all', press=0.0, press_conv_thr=0.5):
    """
    Generate input file for ionic or variable-cell relaxation.
    
    Parameters
    ----------
    prefix : str
        Prefix for output files
    ecutwfc : float
        Wavefunction cutoff in Ry
    ecutrho : float
        Charge density cutoff in Ry  
    kpoints : tuple
        (kx, ky, kz) Monkhorst-Pack grid
    pseudo_dir : str or Path
        Path to pseudopotential directory
    celldm1 : float
        Lattice parameter in Bohr
    calculation : str
        'relax' for ionic only, 'vc-relax' for variable cell
    etot_conv_thr : float
        Energy convergence threshold (Ry)
    forc_conv_thr : float
        Force convergence threshold (Ry/Bohr)
    cell_dofree : str
        Cell degrees of freedom (for vc-relax)
    press : float
        Target pressure in kbar (for vc-relax)
    press_conv_thr : float
        Pressure convergence threshold in kbar (for vc-relax)
    
    Returns
    -------
    str
        Complete input file content
    """
    kx, ky, kz = kpoints
    
    # Build CONTROL namelist
    control = f"""&CONTROL
    calculation = '{calculation}'
    prefix = '{prefix}'
    outdir = './tmp'
    pseudo_dir = '{pseudo_dir}'
    verbosity = 'high'
    tprnfor = .true.
    tstress = .true.
    etot_conv_thr = {etot_conv_thr}
    forc_conv_thr = {forc_conv_thr}
/"""
    
    # Build SYSTEM namelist
    system = f"""&SYSTEM
    ibrav = 2
    celldm(1) = {celldm1}
    nat = 2
    ntyp = 1
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'smearing'
    smearing = 'cold'
    degauss = 0.01
/"""
    
    # Build ELECTRONS namelist
    electrons = f"""&ELECTRONS
    conv_thr = 1.0e-8
    mixing_beta = 0.7
/"""
    
    # Build IONS namelist (required for relax and vc-relax)
    ions = """&IONS
    ion_dynamics = 'bfgs'
/"""
    
    # Build CELL namelist (only for vc-relax)
    if calculation == 'vc-relax':
        cell = f"""&CELL
    cell_dynamics = 'bfgs'
    cell_dofree = '{cell_dofree}'
    press = {press}
    press_conv_thr = {press_conv_thr}
/"""
    else:
        cell = ""
    
    # Build cards
    cards = f"""ATOMIC_SPECIES
    Si  28.0855  Si.upf

ATOMIC_POSITIONS {{crystal}}
    Si  0.00  0.00  0.00
    Si  0.25  0.25  0.25

K_POINTS {{automatic}}
    {kx} {ky} {kz} 0 0 0
"""
    
    return control + "\n" + system + "\n" + electrons + "\n" + ions + "\n" + cell + "\n" + cards

In [ ]:
def generate_scf_input(prefix, ecutwfc, ecutrho, kpoints, pseudo_dir,
                       celldm1, conv_thr=1.0e-8):
    """
    Generate SCF input file for single-point energy calculation.
    
    Parameters
    ----------
    prefix : str
        Prefix for output files
    ecutwfc : float
        Wavefunction cutoff in Ry
    ecutrho : float
        Charge density cutoff in Ry
    kpoints : tuple
        (kx, ky, kz) Monkhorst-Pack grid
    pseudo_dir : str or Path
        Path to pseudopotential directory
    celldm1 : float
        Lattice parameter in Bohr
    conv_thr : float
        SCF convergence threshold in Ry
    
    Returns
    -------
    str
        Complete input file content
    """
    kx, ky, kz = kpoints
    
    input_text = f"""&CONTROL
    calculation = 'scf'
    prefix = '{prefix}'
    outdir = './tmp'
    pseudo_dir = '{pseudo_dir}'
    verbosity = 'high'
    tprnfor = .true.
    tstress = .true.
/

&SYSTEM
    ibrav = 2
    celldm(1) = {celldm1}
    nat = 2
    ntyp = 1
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'smearing'
    smearing = 'cold'
    degauss = 0.01
/

&ELECTRONS
    conv_thr = {conv_thr}
    mixing_beta = 0.7
/

ATOMIC_SPECIES
    Si  28.0855  Si.upf

ATOMIC_POSITIONS {{crystal}}
    Si  0.00  0.00  0.00
    Si  0.25  0.25  0.25

K_POINTS {{automatic}}
    {kx} {ky} {kz} 0 0 0
"""
    return input_text

In [ ]:
def run_pwscf(input_file, timeout=600):
    """
    Run pw.x calculation.
    
    Parameters
    ----------
    input_file : str or Path
        Path to input file
    timeout : int
        Maximum runtime in seconds
    
    Returns
    -------
    tuple
        (output_text, elapsed_time)
    """
    input_file = Path(input_file)
    output_file = input_file.with_suffix('.out')
    work_dir = input_file.parent
    
    start = time.time()
    result = subprocess.run(
        ['pw.x', '-in', input_file.name],
        capture_output=True,
        text=True,
        cwd=work_dir,
        timeout=timeout
    )
    elapsed = time.time() - start
    
    with open(output_file, 'w') as f:
        f.write(result.stdout)
    
    return result.stdout, elapsed

In [ ]:
def parse_scf_output(output_text):
    """
    Parse energy, volume, pressure, and forces from SCF output.
    
    Parameters
    ----------
    output_text : str
        Content of the QE output file
    
    Returns
    -------
    dict
        Dictionary with parsed quantities
    """
    results = {}
    
    # Parse total energy (final converged value)
    for line in output_text.split('\n'):
        if '!' in line and 'total energy' in line:
            match = re.search(r'=\s+([\d.E+-]+)\s+Ry', line)
            if match:
                results['energy_ry'] = float(match.group(1))
        
        # Parse volume
        if 'unit-cell volume' in line:
            match = re.search(r'=\s+([\d.]+)', line)
            if match:
                results['volume_bohr3'] = float(match.group(1))
        
        # Parse pressure
        if 'total   stress' in line and 'P=' in line:
            match = re.search(r'P=\s*([\d.E+-]+)', line)
            if match:
                results['pressure_kbar'] = float(match.group(1))
        
        # Parse total force
        if 'Total force' in line:
            match = re.search(r'Total force\s*=\s*([\d.E+-]+)', line)
            if match:
                results['total_force'] = float(match.group(1))
        
        # Parse lattice parameter
        if 'lattice parameter (alat)' in line:
            match = re.search(r'=\s+([\d.]+)', line)
            if match:
                results['alat_bohr'] = float(match.group(1))
    
    # Check convergence
    results['converged'] = 'convergence has been achieved' in output_text
    
    return results

In [ ]:
def parse_relax_output(output_text):
    """
    Parse relaxation output including optimization trajectory.
    
    Parameters
    ----------
    output_text : str
        Content of the QE output file
    
    Returns
    -------
    dict
        Dictionary with parsed quantities including trajectory
    """
    results = parse_scf_output(output_text)
    
    # Parse BFGS steps
    bfgs_energies = []
    bfgs_forces = []
    bfgs_pressures = []
    
    lines = output_text.split('\n')
    for i, line in enumerate(lines):
        # Parse BFGS step data
        if 'number of bfgs steps' in line:
            match = re.search(r'=\s*(\d+)', line)
            if match:
                results['n_bfgs_steps'] = int(match.group(1))
        
        # Energy at each BFGS step
        if 'total energy' in line and '!' not in line and 'estimated' not in line:
            match = re.search(r'=\s+([\d.E+-]+)\s+Ry', line)
            if match:
                bfgs_energies.append(float(match.group(1)))
        
        # Total force at each step
        if 'Total force' in line:
            match = re.search(r'=\s+([\d.E+-]+)', line)
            if match:
                bfgs_forces.append(float(match.group(1)))
    
    results['bfgs_energies'] = bfgs_energies
    results['bfgs_forces'] = bfgs_forces
    
    # Check if optimization converged
    results['bfgs_converged'] = 'bfgs converged' in output_text.lower() or \
                                 'final scf calculation' in output_text.lower()
    
    return results

---

## 5. Ionic Relaxation (`relax`)

Let's demonstrate ionic relaxation by slightly displacing atoms from equilibrium.

In [ ]:
def generate_displaced_relax_input(prefix, ecutwfc, ecutrho, kpoints, pseudo_dir,
                                    celldm1, displacement=0.02):
    """
    Generate relax input with displaced atomic positions.
    
    Parameters
    ----------
    displacement : float
        Fractional displacement of second atom
    """
    kx, ky, kz = kpoints
    
    # Displace second Si atom from (0.25, 0.25, 0.25)
    pos2 = 0.25 + displacement
    
    input_text = f"""&CONTROL
    calculation = 'relax'
    prefix = '{prefix}'
    outdir = './tmp'
    pseudo_dir = '{pseudo_dir}'
    verbosity = 'high'
    tprnfor = .true.
    tstress = .true.
    etot_conv_thr = 1.0e-5
    forc_conv_thr = 1.0e-4
/

&SYSTEM
    ibrav = 2
    celldm(1) = {celldm1}
    nat = 2
    ntyp = 1
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'smearing'
    smearing = 'cold'
    degauss = 0.01
/

&ELECTRONS
    conv_thr = 1.0e-8
    mixing_beta = 0.7
/

&IONS
    ion_dynamics = 'bfgs'
/

ATOMIC_SPECIES
    Si  28.0855  Si.upf

ATOMIC_POSITIONS {{crystal}}
    Si  0.00  0.00  0.00
    Si  {pos2:.4f}  {pos2:.4f}  {pos2:.4f}

K_POINTS {{automatic}}
    {kx} {ky} {kz} 0 0 0
"""
    return input_text

# Display example input
example_relax = generate_displaced_relax_input(
    prefix='si_relax',
    ecutwfc=ecutwfc,
    ecutrho=ecutrho,
    kpoints=(kgrid, kgrid, kgrid),
    pseudo_dir=PSEUDO_DIR,
    celldm1=10.26,
    displacement=0.02
)

print("Example 'relax' Input File (with displaced atom):")
print("=" * 60)
print(example_relax)

In [ ]:
# Run ionic relaxation (uncomment to execute)
# Note: This requires QE to be installed

print("Ionic Relaxation Demonstration")
print("=" * 60)
print("\nKey parameters for 'relax':")
print("  - calculation = 'relax'")
print("  - etot_conv_thr = 1.0e-5 Ry (energy convergence)")
print("  - forc_conv_thr = 1.0e-4 Ry/Bohr (force convergence)")
print("  - ion_dynamics = 'bfgs' (quasi-Newton optimizer)")
print("\nThe BFGS algorithm:")
print("  1. Computes forces on atoms via Hellmann-Feynman theorem")
print("  2. Updates positions to minimize total energy")
print("  3. Builds approximate Hessian from force history")
print("  4. Converges when forces < forc_conv_thr on all atoms")

# Uncomment below to run actual calculation:
# relax_input_file = WORK_DIR / 'si_relax.in'
# with open(relax_input_file, 'w') as f:
#     f.write(example_relax)
# output, elapsed = run_pwscf(relax_input_file)
# results = parse_relax_output(output)
# print(f"\nRelaxation completed in {elapsed:.1f}s")
# print(f"BFGS steps: {results.get('n_bfgs_steps', 'N/A')}")
# print(f"Final force: {results.get('total_force', 'N/A')} Ry/Bohr")

---

## 6. Variable-Cell Relaxation (`vc-relax`)

Variable-cell relaxation optimizes both atomic positions and cell parameters simultaneously.

In [ ]:
# Generate vc-relax input
vc_relax_input = generate_relax_input(
    prefix='si_vcrelax',
    ecutwfc=ecutwfc,
    ecutrho=ecutrho,
    kpoints=(kgrid, kgrid, kgrid),
    pseudo_dir=PSEUDO_DIR,
    celldm1=10.26,
    calculation='vc-relax',
    cell_dofree='ibrav',  # Maintain cubic symmetry
    press=0.0,            # Target zero pressure
    press_conv_thr=0.5    # Pressure tolerance in kbar
)

print("Example 'vc-relax' Input File:")
print("=" * 60)
print(vc_relax_input)

In [ ]:
print("Variable-Cell Relaxation Parameters")
print("=" * 60)
print("\n&CELL namelist options:")
print("\n  cell_dynamics = 'bfgs'")
print("    - BFGS algorithm for cell optimization")
print("    - Other options: 'damp-pr', 'damp-w' (damped dynamics)")
print("\n  cell_dofree = 'ibrav'")
print("    - Maintains symmetry defined by ibrav")
print("    - For Si (ibrav=2, FCC): only celldm(1) changes")
print("    - Use 'all' for general triclinic optimization")
print("\n  press = 0.0")
print("    - Target external pressure in kbar")
print("    - Use non-zero for high-pressure studies")
print("\n  press_conv_thr = 0.5")
print("    - Pressure convergence threshold in kbar")
print("    - Tighter values (0.1-0.5) for accurate results")

print("\n" + "=" * 60)
print("Interpreting the Stress Tensor:")
print("=" * 60)
print("""
The stress tensor output looks like:

  total   stress  (Ry/bohr**3)    (kbar)     P=   XX.XX
     0.000123   0.000000   0.000000       XX.XX    0.00    0.00
     0.000000   0.000123   0.000000        0.00   XX.XX    0.00
     0.000000   0.000000   0.000123        0.00    0.00   XX.XX

Interpretation:
  - Diagonal elements: normal stresses (expansion/compression)
  - Off-diagonal elements: shear stresses
  - P (pressure) = -Trace(stress)/3
  - Positive P: cell wants to EXPAND (lattice too small)
  - Negative P: cell wants to CONTRACT (lattice too large)
  - |P| < 0.5 kbar typically indicates good convergence
""")

---

## 7. Equation of State (EOS) Method

### Why EOS is More Robust Than `vc-relax`

The EOS method has several advantages:

1. **More robust**: Avoids potential vc-relax convergence issues
2. **Additional properties**: Directly yields bulk modulus $B_0$ and its pressure derivative $B_0'$
3. **Quality check**: Visual inspection of E(V) curve reveals any problems
4. **Better for metals**: vc-relax can be unstable for metals

### The Birch-Murnaghan Equation of State

The third-order Birch-Murnaghan EOS relates energy to volume:

$$E(V) = E_0 + \frac{9V_0B_0}{16}\left[(\eta-1)^3 B_0' + (\eta-1)^2(6-4\eta)\right]$$

where $\eta = \left(\frac{V_0}{V}\right)^{2/3}$

**Parameters:**
- $E_0$: Equilibrium energy
- $V_0$: Equilibrium volume
- $B_0$: Bulk modulus (incompressibility)
- $B_0'$: Pressure derivative of bulk modulus (typically 3-5)

In [ ]:
def birch_murnaghan(V, E0, V0, B0, B0_prime):
    """
    Third-order Birch-Murnaghan equation of state.
    
    Parameters
    ----------
    V : array_like
        Volume(s) at which to evaluate energy
    E0 : float
        Equilibrium energy
    V0 : float
        Equilibrium volume
    B0 : float
        Bulk modulus (same units as E0/V0)
    B0_prime : float
        Pressure derivative of bulk modulus (dimensionless)
    
    Returns
    -------
    E : array_like
        Energy at the given volume(s)
    
    Notes
    -----
    The equation is:
    E(V) = E0 + (9*V0*B0/16) * [(eta-1)^3 * B0' + (eta-1)^2 * (6-4*eta)]
    where eta = (V0/V)^(2/3)
    """
    V = np.asarray(V)
    eta = (V0 / V) ** (2.0 / 3.0)
    
    term1 = (eta - 1.0) ** 3 * B0_prime
    term2 = (eta - 1.0) ** 2 * (6.0 - 4.0 * eta)
    
    E = E0 + (9.0 * V0 * B0 / 16.0) * (term1 + term2)
    
    return E


def birch_murnaghan_pressure(V, V0, B0, B0_prime):
    """
    Pressure from Birch-Murnaghan EOS: P = -dE/dV.
    
    Parameters
    ----------
    V : array_like
        Volume(s)
    V0 : float
        Equilibrium volume
    B0 : float
        Bulk modulus
    B0_prime : float
        Pressure derivative of bulk modulus
    
    Returns
    -------
    P : array_like
        Pressure at the given volume(s)
    """
    V = np.asarray(V)
    eta = (V0 / V) ** (2.0 / 3.0)
    
    P = (3.0 * B0 / 2.0) * (eta ** (7.0 / 2.0) - eta ** (5.0 / 2.0)) * \
        (1.0 + 0.75 * (B0_prime - 4.0) * (eta - 1.0))
    
    return P

In [ ]:
def run_eos_calculations(celldm1_values, ecutwfc, ecutrho, kpoints, 
                          pseudo_dir, work_dir, verbose=True):
    """
    Run SCF calculations for multiple lattice parameters to generate E(V) curve.
    
    Parameters
    ----------
    celldm1_values : array_like
        Array of lattice parameters in Bohr
    ecutwfc : float
        Wavefunction cutoff in Ry
    ecutrho : float
        Charge density cutoff in Ry
    kpoints : tuple
        (kx, ky, kz) k-point grid
    pseudo_dir : str or Path
        Pseudopotential directory
    work_dir : str or Path
        Working directory for calculations
    verbose : bool
        Print progress information
    
    Returns
    -------
    list
        List of dictionaries with results for each calculation
    """
    work_dir = Path(work_dir)
    results = []
    
    if verbose:
        print("\nRunning E(V) calculations...")
        print("=" * 90)
        header = f"{'celldm(1) (Bohr)':<18} {'a (Ang)':<12} {'V (Ang^3)':<12} {'E (Ry)':<18} {'P (kbar)':<12} {'Status'}"
        print(header)
        print("-" * 90)
    
    for celldm1 in celldm1_values:
        # Create unique prefix
        prefix = f'si_a{celldm1:.3f}'.replace('.', 'p')
        
        # Generate input
        input_text = generate_scf_input(
            prefix=prefix,
            ecutwfc=ecutwfc,
            ecutrho=ecutrho,
            kpoints=kpoints,
            pseudo_dir=pseudo_dir,
            celldm1=celldm1
        )
        
        # Write input file
        input_file = work_dir / f'{prefix}.in'
        with open(input_file, 'w') as f:
            f.write(input_text)
        
        # Run calculation
        output, elapsed = run_pwscf(input_file)
        parsed = parse_scf_output(output)
        
        # Convert units
        a_angstrom = celldm1 * 0.529177
        v_bohr3 = parsed.get('volume_bohr3', 0)
        v_angstrom3 = v_bohr3 * 0.529177**3 if v_bohr3 else None
        
        # Store results
        result = {
            'celldm1_bohr': celldm1,
            'a_angstrom': a_angstrom,
            'volume_bohr3': v_bohr3,
            'volume_angstrom3': v_angstrom3,
            'energy_ry': parsed.get('energy_ry'),
            'pressure_kbar': parsed.get('pressure_kbar'),
            'converged': parsed.get('converged', False),
            'elapsed': elapsed
        }
        results.append(result)
        
        # Print progress
        if verbose:
            status = 'OK' if result['converged'] else 'FAILED'
            e_str = f"{result['energy_ry']:.8f}" if result['energy_ry'] else 'N/A'
            p_str = f"{result['pressure_kbar']:.2f}" if result['pressure_kbar'] is not None else 'N/A'
            v_str = f"{v_angstrom3:.4f}" if v_angstrom3 else 'N/A'
            print(f"{celldm1:<18.4f} {a_angstrom:<12.4f} {v_str:<12} {e_str:<18} {p_str:<12} {status}")
    
    if verbose:
        print("=" * 90)
        n_success = sum(1 for r in results if r['converged'])
        print(f"Completed: {n_success}/{len(results)} calculations converged")
    
    return results

In [ ]:
def fit_eos(volumes, energies, p0=None):
    """
    Fit Birch-Murnaghan EOS to E(V) data.
    
    Parameters
    ----------
    volumes : array_like
        Volumes in Bohr^3
    energies : array_like
        Energies in Ry
    p0 : tuple, optional
        Initial guess (E0, V0, B0, B0_prime)
        B0 should be in Ry/Bohr^3
    
    Returns
    -------
    dict
        Fitted parameters and uncertainties:
        - E0, V0 (Bohr^3), B0 (Ry/Bohr^3 and GPa), B0_prime
        - Uncertainties for each parameter
        - a0 (equilibrium lattice parameter in Bohr and Angstrom)
    """
    volumes = np.asarray(volumes)
    energies = np.asarray(energies)
    
    # Conversion factor: 1 Ry/Bohr^3 = 14710.507 GPa
    ry_bohr3_to_gpa = 14710.507
    
    # Initial guesses if not provided
    if p0 is None:
        E0_guess = energies.min()
        V0_guess = volumes[energies.argmin()]
        B0_guess_gpa = 100.0  # Reasonable guess for most solids
        B0_guess = B0_guess_gpa / ry_bohr3_to_gpa
        B0_prime_guess = 4.0
        p0 = (E0_guess, V0_guess, B0_guess, B0_prime_guess)
    
    # Perform fit
    popt, pcov = curve_fit(birch_murnaghan, volumes, energies, p0=p0, maxfev=10000)
    perr = np.sqrt(np.diag(pcov))
    
    E0, V0, B0, B0_prime = popt
    E0_err, V0_err, B0_err, B0_prime_err = perr
    
    # Convert bulk modulus to GPa
    B0_gpa = B0 * ry_bohr3_to_gpa
    B0_gpa_err = B0_err * ry_bohr3_to_gpa
    
    # Calculate equilibrium lattice parameter
    # For FCC (ibrav=2): V_primitive = a^3 / 4 (2 atoms per primitive cell)
    a0_bohr = (4.0 * V0) ** (1.0 / 3.0)
    a0_angstrom = a0_bohr * 0.529177
    
    # Propagate uncertainty to a0
    da0_dV0 = (1.0 / 3.0) * (4.0 / V0) ** (2.0 / 3.0)
    a0_bohr_err = da0_dV0 * V0_err
    a0_angstrom_err = a0_bohr_err * 0.529177
    
    results = {
        'E0': E0,
        'E0_err': E0_err,
        'V0': V0,
        'V0_err': V0_err,
        'B0_ry_bohr3': B0,
        'B0_ry_bohr3_err': B0_err,
        'B0_gpa': B0_gpa,
        'B0_gpa_err': B0_gpa_err,
        'B0_prime': B0_prime,
        'B0_prime_err': B0_prime_err,
        'a0_bohr': a0_bohr,
        'a0_bohr_err': a0_bohr_err,
        'a0_angstrom': a0_angstrom,
        'a0_angstrom_err': a0_angstrom_err,
        'popt': popt,
        'pcov': pcov
    }
    
    return results

In [ ]:
def plot_eos(volumes, energies, fit_results, pressures=None, save_path=None):
    """
    Create publication-quality plots of the equation of state.
    
    Parameters
    ----------
    volumes : array_like
        Calculated volumes in Bohr^3
    energies : array_like
        Calculated energies in Ry
    fit_results : dict
        Results from fit_eos()
    pressures : array_like, optional
        Calculated pressures in kbar
    save_path : str or Path, optional
        Path to save figure
    
    Returns
    -------
    fig : matplotlib Figure
        The figure object
    """
    volumes = np.asarray(volumes)
    energies = np.asarray(energies)
    
    # Generate smooth curve for fit
    V_smooth = np.linspace(volumes.min() * 0.98, volumes.max() * 1.02, 200)
    E_fit = birch_murnaghan(V_smooth, *fit_results['popt'])
    
    # Convert to lattice parameter for second x-axis
    a_data = (4.0 * volumes) ** (1.0 / 3.0) * 0.529177
    a_smooth = (4.0 * V_smooth) ** (1.0 / 3.0) * 0.529177
    
    # Create figure
    if pressures is not None:
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    else:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        axes = axes.reshape(1, 2)
    
    # Plot 1: E(V) curve
    ax1 = axes[0, 0]
    ax1.plot(volumes, energies, 'o', markersize=10, color='#1f77b4', 
             markeredgecolor='black', markeredgewidth=1, label='DFT data')
    ax1.plot(V_smooth, E_fit, '-', linewidth=2, color='#d62728', 
             label='Birch-Murnaghan fit')
    ax1.axvline(x=fit_results['V0'], color='#2ca02c', linestyle='--', 
                alpha=0.7, linewidth=1.5,
                label=f"$V_0$ = {fit_results['V0']:.2f} Bohr$^3$")
    ax1.set_xlabel('Volume (Bohr$^3$)')
    ax1.set_ylabel('Total Energy (Ry)')
    ax1.set_title('Energy vs Volume')
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: E(a) curve
    ax2 = axes[0, 1]
    ax2.plot(a_data, energies, 'o', markersize=10, color='#1f77b4',
             markeredgecolor='black', markeredgewidth=1, label='DFT data')
    ax2.plot(a_smooth, E_fit, '-', linewidth=2, color='#d62728',
             label='Birch-Murnaghan fit')
    ax2.axvline(x=fit_results['a0_angstrom'], color='#2ca02c', linestyle='--',
                alpha=0.7, linewidth=1.5,
                label=f"$a_0$ = {fit_results['a0_angstrom']:.4f} Ang")
    # Mark experimental value for silicon
    ax2.axvline(x=5.431, color='#ff7f0e', linestyle=':', alpha=0.7,
                linewidth=1.5, label='Expt. 5.431 Ang')
    ax2.set_xlabel('Lattice Parameter (Angstrom)')
    ax2.set_ylabel('Total Energy (Ry)')
    ax2.set_title('Energy vs Lattice Parameter')
    ax2.legend(loc='upper right')
    ax2.grid(True, alpha=0.3)
    
    if pressures is not None:
        pressures = np.asarray(pressures)
        
        # Calculate fitted pressure
        P_fit_gpa = birch_murnaghan_pressure(V_smooth, fit_results['V0'],
                                             fit_results['B0_gpa'],
                                             fit_results['B0_prime'])
        P_fit_kbar = P_fit_gpa * 10.0
        
        # Plot 3: P(V) curve
        ax3 = axes[1, 0]
        ax3.plot(volumes, pressures, 'o', markersize=10, color='#1f77b4',
                 markeredgecolor='black', markeredgewidth=1, label='DFT data')
        ax3.plot(V_smooth, P_fit_kbar, '-', linewidth=2, color='#d62728',
                 label='From EOS fit')
        ax3.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        ax3.axvline(x=fit_results['V0'], color='#2ca02c', linestyle='--',
                    alpha=0.7, linewidth=1.5, label='$V_0$ (P=0)')
        ax3.set_xlabel('Volume (Bohr$^3$)')
        ax3.set_ylabel('Pressure (kbar)')
        ax3.set_title('Pressure vs Volume')
        ax3.legend(loc='upper right')
        ax3.grid(True, alpha=0.3)
        
        # Plot 4: Fit residuals
        ax4 = axes[1, 1]
        E_fit_at_data = birch_murnaghan(volumes, *fit_results['popt'])
        residuals = (energies - E_fit_at_data) * 1000  # Convert to mRy
        ax4.bar(range(len(residuals)), residuals, color='#1f77b4', 
                edgecolor='black', alpha=0.7)
        ax4.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        ax4.set_xlabel('Data Point Index')
        ax4.set_ylabel('Residual (mRy)')
        ax4.set_title('Fit Residuals')
        ax4.grid(True, alpha=0.3, axis='y')
        
        # Add RMSE annotation
        rmse = np.sqrt(np.mean(residuals**2))
        ax4.annotate(f'RMSE = {rmse:.3f} mRy', xy=(0.95, 0.95),
                     xycoords='axes fraction', ha='right', va='top',
                     fontsize=11, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(str(save_path), dpi=150, bbox_inches='tight')
        print(f"Figure saved to: {save_path}")
    
    return fig

---

## 8. Example: Silicon Lattice Optimization via EOS

Let's perform a complete EOS calculation for silicon.

In [ ]:
# Define lattice parameter range for EOS scan
# Experimental Si: a = 5.431 Angstrom = 10.263 Bohr
a_exp_angstrom = 5.431
a_exp_bohr = a_exp_angstrom / 0.529177

# Scan +/- 4% around experimental value with 11 points
n_points = 11
scan_range = 0.04  # 4%
celldm1_values = np.linspace(a_exp_bohr * (1 - scan_range), 
                             a_exp_bohr * (1 + scan_range), 
                             n_points)

print("Silicon Equation of State Calculation")
print("=" * 60)
print(f"\nExperimental lattice parameter:")
print(f"  a = {a_exp_angstrom} Angstrom = {a_exp_bohr:.4f} Bohr")
print(f"\nScan parameters:")
print(f"  Range: +/- {scan_range*100:.1f}%")
print(f"  Points: {n_points}")
print(f"  celldm(1) range: {celldm1_values[0]:.4f} - {celldm1_values[-1]:.4f} Bohr")
print(f"  a range: {celldm1_values[0]*0.529177:.4f} - {celldm1_values[-1]*0.529177:.4f} Angstrom")

In [ ]:
# Run EOS calculations
# NOTE: Uncomment below to run actual calculations (requires QE installed)

# eos_results = run_eos_calculations(
#     celldm1_values=celldm1_values,
#     ecutwfc=ecutwfc,
#     ecutrho=ecutrho,
#     kpoints=(kgrid, kgrid, kgrid),
#     pseudo_dir=PSEUDO_DIR,
#     work_dir=WORK_DIR,
#     verbose=True
# )

# For demonstration, use simulated data based on typical PBE results
print("\nUsing simulated data for demonstration...")
print("(Uncomment the code above to run actual calculations)\n")

# Simulated results (typical PBE values for Si)
eos_results = []
V0_sim = 270.0  # Typical equilibrium volume for Si with PBE
E0_sim = -15.85  # Typical Si energy
B0_sim = 0.00635  # ~93 GPa in Ry/Bohr^3
B0p_sim = 4.2

for celldm1 in celldm1_values:
    a_ang = celldm1 * 0.529177
    V = celldm1**3 / 4.0  # FCC primitive cell volume
    E = birch_murnaghan(V, E0_sim, V0_sim, B0_sim, B0p_sim)
    # Add small random noise to simulate real data
    E += np.random.normal(0, 0.0001)
    P = birch_murnaghan_pressure(V, V0_sim, B0_sim * 14710.507, B0p_sim) * 10  # kbar
    
    eos_results.append({
        'celldm1_bohr': celldm1,
        'a_angstrom': a_ang,
        'volume_bohr3': V,
        'volume_angstrom3': V * 0.529177**3,
        'energy_ry': E,
        'pressure_kbar': P,
        'converged': True
    })

# Display results table
print("E(V) Calculation Results:")
print("=" * 90)
print(f"{'celldm(1) (Bohr)':<18} {'a (Ang)':<12} {'V (Ang^3)':<12} {'E (Ry)':<18} {'P (kbar)':<12}")
print("-" * 90)
for r in eos_results:
    print(f"{r['celldm1_bohr']:<18.4f} {r['a_angstrom']:<12.4f} {r['volume_angstrom3']:<12.4f} {r['energy_ry']:<18.8f} {r['pressure_kbar']:<12.2f}")
print("=" * 90)

In [ ]:
# Extract data for fitting
converged_data = [r for r in eos_results if r['converged'] and r['energy_ry'] is not None]

V_data = np.array([r['volume_bohr3'] for r in converged_data])
E_data = np.array([r['energy_ry'] for r in converged_data])
P_data = np.array([r['pressure_kbar'] for r in converged_data])
a_data = np.array([r['celldm1_bohr'] for r in converged_data])

print(f"Extracted {len(converged_data)} converged data points for fitting")

In [ ]:
# Fit the Birch-Murnaghan equation of state
fit_results = fit_eos(V_data, E_data)

print("\nBirch-Murnaghan Equation of State Fit Results")
print("=" * 60)
print(f"\nEquilibrium energy:")
print(f"  E0 = {fit_results['E0']:.8f} +/- {fit_results['E0_err']:.8f} Ry")
print(f"\nEquilibrium volume:")
print(f"  V0 = {fit_results['V0']:.4f} +/- {fit_results['V0_err']:.4f} Bohr^3")
print(f"     = {fit_results['V0'] * 0.529177**3:.4f} Angstrom^3")
print(f"\nBulk modulus:")
print(f"  B0 = {fit_results['B0_gpa']:.2f} +/- {fit_results['B0_gpa_err']:.2f} GPa")
print(f"\nBulk modulus pressure derivative:")
print(f"  B0' = {fit_results['B0_prime']:.2f} +/- {fit_results['B0_prime_err']:.2f}")
print(f"\nEquilibrium lattice parameter:")
print(f"  a0 = {fit_results['a0_bohr']:.4f} +/- {fit_results['a0_bohr_err']:.4f} Bohr")
print(f"     = {fit_results['a0_angstrom']:.4f} +/- {fit_results['a0_angstrom_err']:.4f} Angstrom")
print("=" * 60)

In [ ]:
# Compare with experimental values
a_exp = 5.431   # Angstrom
B0_exp = 98.8   # GPa (experimental bulk modulus)

a_error = (fit_results['a0_angstrom'] - a_exp) / a_exp * 100
B_error = (fit_results['B0_gpa'] - B0_exp) / B0_exp * 100

print("\nComparison with Experimental Values (Silicon)")
print("=" * 60)
print(f"{'Property':<25} {'DFT (PBE)':<15} {'Experiment':<15} {'Error'}")
print("-" * 60)
print(f"{'Lattice parameter (Ang)':<25} {fit_results['a0_angstrom']:<15.4f} {a_exp:<15.3f} {a_error:+.2f}%")
print(f"{'Bulk modulus (GPa)':<25} {fit_results['B0_gpa']:<15.2f} {B0_exp:<15.1f} {B_error:+.2f}%")
print(f"{'B0 prime':<25} {fit_results['B0_prime']:<15.2f} {'~4.2':<15} {'--'}")
print("=" * 60)
print("\nNotes:")
print("  - PBE typically OVERestimates lattice parameters by 1-2%")
print("  - PBE typically UNDERestimates bulk modulus by 5-10%")
print("  - LDA shows opposite trends")

In [ ]:
# Create publication-quality plots
fig = plot_eos(V_data, E_data, fit_results, pressures=P_data,
               save_path=WORK_DIR / 'silicon_eos.png')
plt.show()

---

## 9. Practical Workflow for Structure Optimization

### Recommended Workflow

```
Step 1: Convergence Tests (at initial structure)
        |-- ecutwfc convergence
        |-- k-point convergence
        v
Step 2: Volume Optimization
        |-- Option A: EOS fitting (recommended)
        |-- Option B: vc-relax with cell_dofree='ibrav'
        v
Step 3: Ionic Relaxation (at optimal volume)
        |-- relax calculation
        v
Step 4: Verification
        |-- Check forces ~ 0
        |-- Check pressure ~ 0
        v
Step 5: Production Calculations
```

### When to Use Each Method

| Situation | Recommended Approach |
|-----------|---------------------|
| Simple materials (Si, metals) | EOS fitting |
| Complex structures | vc-relax first, then verify with EOS |
| Need bulk modulus | EOS fitting (mandatory) |
| High-pressure studies | EOS + vc-relax at target pressure |
| Unknown structure | vc-relax with cell_dofree='all' |
| Fixed cell, optimize atoms | relax only |

---

## 10. Extracting and Saving Optimized Structure

In [ ]:
def extract_optimized_structure(output_text):
    """
    Extract final atomic positions and cell parameters from relaxation output.
    
    Parameters
    ----------
    output_text : str
        Content of QE output file from relax or vc-relax calculation
    
    Returns
    -------
    dict
        Dictionary containing:
        - 'cell_parameters': 3x3 array of cell vectors (if found)
        - 'atomic_positions': list of (symbol, x, y, z) tuples
        - 'position_units': 'crystal', 'angstrom', etc.
        - 'alat': lattice parameter if ibrav != 0
    """
    lines = output_text.split('\n')
    results = {}
    
    # Find FINAL coordinates (last occurrence of ATOMIC_POSITIONS)
    positions = []
    in_positions = False
    position_units = None
    
    # Find final cell parameters
    cell_vectors = []
    in_cell = False
    
    # We want the LAST occurrence (final structure), so we scan the whole file
    for i, line in enumerate(lines):
        # Parse CELL_PARAMETERS
        if 'CELL_PARAMETERS' in line:
            in_cell = True
            cell_vectors = []
            continue
        
        if in_cell:
            parts = line.split()
            if len(parts) == 3:
                cell_vectors.append([float(x) for x in parts])
                if len(cell_vectors) == 3:
                    in_cell = False
            elif len(parts) != 0:
                in_cell = False
        
        # Parse ATOMIC_POSITIONS
        if 'ATOMIC_POSITIONS' in line:
            in_positions = True
            positions = []
            # Extract units
            if 'crystal' in line.lower():
                position_units = 'crystal'
            elif 'angstrom' in line.lower():
                position_units = 'angstrom'
            elif 'bohr' in line.lower():
                position_units = 'bohr'
            else:
                position_units = 'alat'
            continue
        
        if in_positions:
            parts = line.split()
            if len(parts) >= 4:
                symbol = parts[0]
                x, y, z = float(parts[1]), float(parts[2]), float(parts[3])
                positions.append((symbol, x, y, z))
            elif len(parts) == 0 or 'End' in line:
                in_positions = False
        
        # Parse final alat (for ibrav != 0 cases)
        if 'lattice parameter (alat)' in line:
            match = re.search(r'=\s+([\d.]+)', line)
            if match:
                results['alat'] = float(match.group(1))
    
    if cell_vectors:
        results['cell_parameters'] = np.array(cell_vectors)
    
    if positions:
        results['atomic_positions'] = positions
        results['position_units'] = position_units
    
    return results

In [ ]:
def write_optimized_input(structure, ecutwfc, ecutrho, kpoints, pseudo_dir,
                          prefix='optimized', calculation='scf'):
    """
    Write a new QE input file using optimized structure.
    
    Parameters
    ----------
    structure : dict
        Output from extract_optimized_structure()
    ecutwfc, ecutrho : float
        Cutoff energies
    kpoints : tuple
        K-point grid
    pseudo_dir : str or Path
        Pseudopotential directory
    prefix : str
        Job prefix
    calculation : str
        Calculation type
    
    Returns
    -------
    str
        Complete input file content
    """
    kx, ky, kz = kpoints
    positions = structure.get('atomic_positions', [])
    units = structure.get('position_units', 'crystal')
    nat = len(positions)
    
    # Get unique species
    species = list(set([p[0] for p in positions]))
    ntyp = len(species)
    
    # Build input
    input_lines = []
    
    input_lines.append(f"""&CONTROL
    calculation = '{calculation}'
    prefix = '{prefix}'
    outdir = './tmp'
    pseudo_dir = '{pseudo_dir}'
    verbosity = 'high'
    tprnfor = .true.
    tstress = .true.
/""")
    
    # System namelist
    if 'cell_parameters' in structure:
        # Use ibrav=0 with explicit cell
        input_lines.append(f"""\n&SYSTEM
    ibrav = 0
    nat = {nat}
    ntyp = {ntyp}
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'smearing'
    smearing = 'cold'
    degauss = 0.01
/""")
    else:
        # Use ibrav=2 (FCC) with celldm(1)
        alat = structure.get('alat', 10.26)
        input_lines.append(f"""\n&SYSTEM
    ibrav = 2
    celldm(1) = {alat}
    nat = {nat}
    ntyp = {ntyp}
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'smearing'
    smearing = 'cold'
    degauss = 0.01
/""")
    
    input_lines.append("""\n&ELECTRONS
    conv_thr = 1.0e-8
    mixing_beta = 0.7
/""")
    
    # Cell parameters card (if ibrav=0)
    if 'cell_parameters' in structure:
        cell = structure['cell_parameters']
        input_lines.append("\nCELL_PARAMETERS {angstrom}")
        for row in cell:
            input_lines.append(f"  {row[0]:16.10f}  {row[1]:16.10f}  {row[2]:16.10f}")
    
    # Atomic species
    input_lines.append("\nATOMIC_SPECIES")
    # Simple mapping for common elements
    masses = {'Si': 28.0855, 'C': 12.011, 'O': 15.999, 'N': 14.007, 'H': 1.008}
    for sp in species:
        mass = masses.get(sp, 1.0)
        input_lines.append(f"    {sp}  {mass}  {sp}.upf")
    
    # Atomic positions
    input_lines.append(f"\nATOMIC_POSITIONS {{{units}}}")
    for symbol, x, y, z in positions:
        input_lines.append(f"    {symbol}  {x:14.10f}  {y:14.10f}  {z:14.10f}")
    
    # K-points
    input_lines.append(f"\nK_POINTS {{automatic}}")
    input_lines.append(f"    {kx} {ky} {kz} 0 0 0")
    
    return '\n'.join(input_lines)

In [ ]:
def write_cif(structure, alat_angstrom, filename):
    """
    Write structure to CIF format for use in other codes.
    
    Parameters
    ----------
    structure : dict
        Structure dictionary with atomic positions
    alat_angstrom : float
        Lattice parameter in Angstrom (for cubic systems)
    filename : str or Path
        Output CIF file path
    """
    positions = structure.get('atomic_positions', [])
    
    cif_lines = [
        "data_optimized_structure",
        "_symmetry_space_group_name_H-M   'F d -3 m'",
        "_symmetry_Int_Tables_number      227",
        f"_cell_length_a                   {alat_angstrom:.6f}",
        f"_cell_length_b                   {alat_angstrom:.6f}",
        f"_cell_length_c                   {alat_angstrom:.6f}",
        "_cell_angle_alpha                90.000000",
        "_cell_angle_beta                 90.000000",
        "_cell_angle_gamma                90.000000",
        "",
        "loop_",
        "_atom_site_label",
        "_atom_site_type_symbol",
        "_atom_site_fract_x",
        "_atom_site_fract_y",
        "_atom_site_fract_z",
    ]
    
    for i, (symbol, x, y, z) in enumerate(positions):
        label = f"{symbol}{i+1}"
        cif_lines.append(f"{label}  {symbol}  {x:.6f}  {y:.6f}  {z:.6f}")
    
    with open(filename, 'w') as f:
        f.write('\n'.join(cif_lines))
    
    print(f"CIF file written to: {filename}")

In [ ]:
# Demonstrate saving optimized structure
print("Saving Optimized Structure")
print("=" * 60)

# Create a mock optimized structure based on our EOS results
optimized_structure = {
    'alat': fit_results['a0_bohr'],
    'atomic_positions': [
        ('Si', 0.0, 0.0, 0.0),
        ('Si', 0.25, 0.25, 0.25)
    ],
    'position_units': 'crystal'
}

# Write QE input file
optimized_input = write_optimized_input(
    structure=optimized_structure,
    ecutwfc=ecutwfc,
    ecutrho=ecutrho,
    kpoints=(kgrid, kgrid, kgrid),
    pseudo_dir=PSEUDO_DIR,
    prefix='si_optimized'
)

optimized_file = WORK_DIR / 'si_optimized.in'
with open(optimized_file, 'w') as f:
    f.write(optimized_input)
print(f"QE input file written to: {optimized_file}")

# Write CIF file
cif_file = WORK_DIR / 'si_optimized.cif'
write_cif(optimized_structure, fit_results['a0_angstrom'], cif_file)

print("\nOptimized input file content:")
print("-" * 60)
print(optimized_input)

---

## 11. Save Optimized Parameters

In [ ]:
# Update the converged parameters file
params_file = WORKSHOP_ROOT / 'converged_parameters.json'

if params_file.exists():
    with open(params_file, 'r') as f:
        params = json.load(f)
else:
    params = {}

# Add structure optimization results
params['celldm1_optimized_bohr'] = float(fit_results['a0_bohr'])
params['a_optimized_angstrom'] = float(fit_results['a0_angstrom'])
params['V0_bohr3'] = float(fit_results['V0'])
params['E0_ry'] = float(fit_results['E0'])
params['bulk_modulus_gpa'] = float(fit_results['B0_gpa'])
params['bulk_modulus_derivative'] = float(fit_results['B0_prime'])

with open(params_file, 'w') as f:
    json.dump(params, f, indent=2)

print("Updated converged parameters:")
print(json.dumps(params, indent=2))
print(f"\nSaved to: {params_file}")

---

## Summary

In this notebook, we have covered:

### 1. Why Structure Optimization is Essential
- Database structures are not at DFT equilibrium
- Different functionals give different equilibrium structures
- Must optimize with YOUR methodology before property calculations

### 2. Types of Optimization in Quantum ESPRESSO
- `calculation = 'relax'`: Ionic positions only (fixed cell)
- `calculation = 'vc-relax'`: Variable cell (ions + cell shape + volume)

### 3. Equation of State Method
- More robust than vc-relax for volume optimization
- Provides bulk modulus and its pressure derivative
- Birch-Murnaghan equation: $E(V) = E_0 + \frac{9V_0B_0}{16}[(\eta-1)^3 B_0' + (\eta-1)^2(6-4\eta)]$

### 4. Key Results for Silicon (PBE)

| Property | DFT (PBE) | Experiment | Typical Error |
|----------|-----------|------------|---------------|
| Lattice parameter | ~5.47 Ang | 5.431 Ang | +1-2% |
| Bulk modulus | ~93 GPa | 98.8 GPa | -5-10% |

### 5. Practical Workflow
1. Convergence tests at initial structure
2. Volume optimization (EOS or vc-relax)
3. Ionic relaxation at optimal volume
4. Verify forces ~ 0, pressure ~ 0

### Next Steps
With the optimized structure, you can proceed to:
- **Band Structure Calculations** (Notebook 06)
- **Density of States** (Notebook 07)
- **Phonon Calculations**
- **Optical Properties**